In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [ ]:

class WeightedF1Score(tf.keras.metrics.Metric):
    def __init__(self, name='weighted_f1', class_weights=[0.2, 0.8], **kwargs):
        super().__init__(name=name, **kwargs)
        self.class_weights = tf.constant(class_weights, dtype=tf.float32)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        self.precision.update_state(y_true, y_pred)
        self.recall.update_state(y_true, y_pred)
        
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        f1 = 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))
        return tf.reduce_sum(f1 * self.class_weights)
    
    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()


In [ ]:

np.random.seed(42)
n_samples = 10000
fraud_ratio = 0.05  

X = np.random.randn(n_samples, 5)  
y = np.random.binomial(1, fraud_ratio, n_samples)

X[y == 1] += 1.5  


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)


In [ ]:

model = Sequential([
    Dense(16, activation='relu', input_shape=(5,)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

class_weights = {0: 1, 1: 20}  


In [13]:

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', WeightedF1Score(class_weights=[0.2, 0.8])]
)


In [14]:

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    class_weight=class_weights,
    validation_split=0.2,
    verbose=1
)


Epoch 1/10


175/175 [==============================] - 3s 6ms/step - loss: 1.4145 - accuracy: 0.7577 - weighted_f1: 0.1524 - val_loss: 0.5417 - val_accuracy: 0.7164 - val_weighted_f1: 0.2012
Epoch 2/10
175/175 [==============================] - 1s 3ms/step - loss: 0.6733 - accuracy: 0.7804 - weighted_f1: 0.2841 - val_loss: 0.3484 - val_accuracy: 0.8386 - val_weighted_f1: 0.3110
Epoch 3/10
175/175 [==============================] - 1s 3ms/step - loss: 0.4662 - accuracy: 0.8675 - weighted_f1: 0.3958 - val_loss: 0.2906 - val_accuracy: 0.8643 - val_weighted_f1: 0.3493
Epoch 4/10
175/175 [==============================] - 1s 3ms/step - loss: 0.3955 - accuracy: 0.8888 - weighted_f1: 0.4423 - val_loss: 0.2075 - val_accuracy: 0.9057 - val_weighted_f1: 0.4261
Epoch 5/10
175/175 [==============================] - 1s 3ms/step - loss: 0.3641 - accuracy: 0.9086 - weighted_f1: 0.4890 - val_loss: 0.1997 - val_accuracy: 0.9064 - val_weighted_f1: 0.4279
Epoch 6/10
175/175 [============================

In [ ]:

y_pred = (model.predict(X_test) > 0.5).astype(int)
standard_report = classification_report(y_test, y_pred, output_dict=True)
standard_f1 = standard_report['1']['f1-score']


custom_f1 = history.history['val_weighted_f1'][-1] 


94/94 [==============================] - 0s 2ms/step


In [ ]:

results = pd.DataFrame({
    'Metric': ['Standard F1 (Fraud Class)', 'Custom Weighted F1'],
    'Value': [standard_f1, custom_f1]
})

print("\nMetric Comparison:")
print(results)



Metric Comparison:
                      Metric     Value
0  Standard F1 (Fraud Class)  0.526749
1         Custom Weighted F1  0.468899
